In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
from nltk.corpus import wordnet
syns = wordnet.synsets("program")

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import re
import unicodedata
import sys

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import urllib.request
import urllib.response
import sys
import os, glob
import http.client, urllib
import json
import http.client, urllib.request, urllib.parse, urllib.error, base64

# Replace the accessKey string value with your valid access key.
accessKey = '82e4612615634c398bfd26e7d6327833'
url = 'westcentralus.api.cognitive.microsoft.com'
path = '/text/analytics/v2.0/keyPhrases'

def extract_keywords(body):
    headers = {'Ocp-Apim-Subscription-Key': accessKey}
    conn = http.client.HTTPSConnection(url)
    body_req = {'documents':[{'language': 'en', 'id': 1, 'text': body}]}
    body_json = json.dumps(body_req)
    conn.request ("POST", path, body_json, headers)
    response = conn.getresponse ()
    string = response.read().decode('utf-8')
    json_obj = json.loads(string)
    print("json_obj: ", json_obj)
    keyphrases_list = ((json_obj['documents'])[0])['keyPhrases']
    print("keyphrases_list: ", keyphrases_list)
    return keyphrases_list

In [3]:
# medical conditions training data
training_data = []

import csv

# adds every medical condition along with associated info from csv file
with open('Disease CSV fixed.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count != 0:
            training_data.append({"class":row[1], "synonyms":row[2], 
                                  "sentence":row[3], "treatments":row[4], "danger level":row[5]})
        line_count += 1

print ("%s total classes in training data" % len(training_data))

for term in training_data:
    print(term["class"] + " | " + term["synonyms"] + " | " + term["sentence"] + 
          " | " + term["treatments"] + " | " + term["danger level"])

28 total classes in training data
Common Cold | Upper Respiratory Tract;Nose and Throat Infection | Runny nose;stuffy nose;sore throat;cough;congestion;body aches;sneezing | Stay hydrated;rest;sooth a sore throat using saltwater gargle;take over the counter cold and cough medications
 | 1
Allergies | Allergic Reaction | Eye irritation;runny stuffy nose;puffy watery eyes;sneezing;inflamed;itchy nose; itchy throat | Remove the cause of allergy; 
otherwise take Antihistamines to relieve sneezing;Decongestants to relieve congestions in nasal membranes;Anti inflamary agents to reduce;Allergy shots | 1
Conjunctivitis | Pink Eye | Redness eyes;red eye;itchy eyes;eyeball tearing burning sensation | See a doctor;Bacterial cases can be treated with antibiotic eye drops;Allergic reactions can be treated with other eye drops | 1
Irritable Bowel Syndrome (IBS) | Diarrhea | Watery, loose stools;frequent bowel movement;stomachache nausea;bloody stool | Diarrhea will typically go away in 2 days;Consul

In [4]:
# a table structure to hold the different punctuation used
tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                    if unicodedata.category(chr(i)).startswith('P'))

# method to remove punctuations from sentences.
def remove_punctuation(text):
     return text.translate(tbl)

# method to slice strings
def slicer(my_str,sub):
    index=my_str.find(sub)
    if index !=-1 :
         return my_str[index:] 
    else :
         raise Exception('Sub string not found!')

# find synonyms of all the words in a sentence
def find_syns(sentence):
    words = []
    for word in sentence.split():
        if word != '':
            synonyms = []
            for syn in wordnet.synsets(word):
                for l in syn.lemmas():
                    if l.name() not in synonyms and l.name() != word:
                        synonyms.append(l.name())
            word = (word + ' ' + ' '.join(synonyms)).strip()
            words.append(word)
    print("SYNONYMS:",words)
    return words

# clean all synonyms and words
def clean_syns(arg):
    clean_synonyms = []
    for syn_set in synonyms:
        syn_set = ' '.join(s for s in syn_set.split() if not any(c.isdigit() for c in s))
        syn_set = re.sub(r'\d+', '', syn_set)
        syn_set = syn_set.replace("_"," ")
        syn_set = remove_punctuation(syn_set)
        clean_synonyms.append(syn_set)
    print("CLEAN SYNONYMS: ",clean_synonyms)
    return clean_synonyms

In [5]:
import string

words = []
words_total = []
classes = []
classes_details = []
documents = []
ignore_words = ['?']

# loop through each description per symptom
for pattern in training_data:
    symptoms = pattern['sentence']
    word_list = symptoms.split(';')
    print("WORD_LIST:",word_list)
    word_list_new = []
    for word in word_list:
        word = remove_punctuation(word)
        word = word.strip().lower()
        w = word.split()
        for item in w:
            if item in ["and", "or", "the", "lot", "of", "in", "on", "a", "are", "is", "am", 
                                 "lots", "were", "was", "then", "but", "as", "to", "can","an","from","off"]:
                w.remove(item)
        word = ' '.join(w)
        word_list_new.append(word)
    print("WORD_LIST_NEW:",word_list_new)
    joined_symps = ' '.join(s for s in word_list_new)
    print("joined_symps:",joined_symps)
    
    #find synonyms of each word in description
    synonyms = find_syns(joined_symps)
    synonyms_clean = clean_syns(synonyms)
    
    # tokenize each word in the description
    w_list = []
    for syn_set in synonyms_clean:
        w_token = nltk.word_tokenize(syn_set)
        w = [stemmer.stem(i.lower()) for i in w_token if i not in ignore_words]
        w = sorted(list(set(w)))
    
        # strip duplicates
        for item in w:
            if not item in words_total and item != '':
                words_total.append(item.strip())
            else:
                w.remove(item)
    
        w_syn = ' '.join(w)
        w_list.append(w_syn)
    
    #add each string to words
    for item in w_list:
        if not item in words and item != '':
            #print("Item is in words array already.")
            words.append(item.strip())
     
    # add to documents in our corpus
    documents.append((w_list, pattern['class']))
    
    # add to our classes list
    if pattern['class'] not in classes:
        classes.append(pattern['class'])
        classes_details.append({"class":pattern['class'], "synonyms":pattern['synonyms'],
                                "treatments":pattern['treatments'], "danger level":pattern['danger level']})
        
#remove duplicates in words
words = sorted(list(set(words)))

# remove duplicates in classes
classes = sorted(list(set(classes)))

print (len(documents), "documents", documents)
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)
print ("classes_details", classes_details)

WORD_LIST: ['Runny nose', 'stuffy nose', 'sore throat', 'cough', 'congestion', 'body aches', 'sneezing']
WORD_LIST_NEW: ['runny nose', 'stuffy nose', 'sore throat', 'cough', 'congestion', 'body aches', 'sneezing']
joined_symps: runny nose stuffy nose sore throat cough congestion body aches sneezing
SYNONYMS: ['runny fluid', 'nose olfactory_organ nozzle intrude horn_in pry poke scent wind nuzzle', 'stuffy airless close unaired stodgy', 'nose olfactory_organ nozzle intrude horn_in pry poke scent wind nuzzle', 'sore sensitive raw tender afflictive painful huffy mad', 'throat pharynx', 'cough coughing', 'congestion over-crowding', 'body organic_structure physical_structure dead_body torso trunk consistency consistence eubstance soundbox personify', 'aches ache aching hurt suffer yearn yen pine languish smart', 'sneezing sneeze sternutation']
CLEAN SYNONYMS:  ['runny fluid', 'nose olfactory organ nozzle intrude horn in pry poke scent wind nuzzle', 'stuffy airless close unaired stodgy', 'nos

WORD_LIST: ['fever', 'cough', 'muscle aches', 'headache', 'chills', 'loss appetite', 'chills', 'fatigue', 'sore throat', ' runny nose', ' stuffy nose', ' stomach ache', 'diarrhea', 'vomiting']
WORD_LIST_NEW: ['fever', 'cough', 'muscle aches', 'headache', 'chills', 'loss appetite', 'chills', 'fatigue', 'sore throat', 'runny nose', 'stuffy nose', 'stomach ache', 'diarrhea', 'vomiting']
joined_symps: fever cough muscle aches headache chills loss appetite chills fatigue sore throat runny nose stuffy nose stomach ache diarrhea vomiting
SYNONYMS: ['fever febrility febricity pyrexia feverishness', 'cough coughing', 'muscle musculus muscular_tissue muscleman brawn brawniness muscularity sinew heftiness', 'aches ache aching hurt suffer yearn yen pine languish smart', 'headache concern worry vexation head_ache cephalalgia', 'chills chill iciness gelidity frisson shiver quiver shudder thrill tingle shivering pall cool cool_down', 'loss deprivation red_ink red personnel_casualty passing departure 

SYNONYMS: ['fever febrility febricity pyrexia feverishness', 'cough coughing', 'sore sensitive raw tender afflictive painful huffy mad', 'throat pharynx', 'runny fluid', 'nose olfactory_organ nozzle intrude horn_in pry poke scent wind nuzzle', 'stuffy airless close unaired stodgy', 'nose olfactory_organ nozzle intrude horn_in pry poke scent wind nuzzle', 'watery reeking washy weak', 'eyes eye oculus optic center centre middle heart eyeball', 'red redness Red Red_River Bolshevik Marxist bolshie bolshy loss red_ink reddish ruddy blood-red carmine cerise cherry cherry-red crimson ruby ruby-red scarlet violent reddened red-faced flushed', 'eyes eye oculus optic center centre middle heart eyeball', 'body organic_structure physical_structure dead_body torso trunk consistency consistence eubstance soundbox personify', 'aches ache aching hurt suffer yearn yen pine languish smart', 'headache concern worry vexation head_ache cephalalgia', 'fatigue weariness tiredness fatigue_duty tire pall weary

SYNONYMS: ['fever febrility febricity pyrexia feverishness', 'aching ache hurt suffer yearn yen pine languish smart achy', 'muscles muscle musculus muscular_tissue muscleman brawn brawniness muscularity sinew heftiness', 'loss deprivation red_ink red personnel_casualty passing departure exit expiration going release', 'appetite appetency appetence', 'rash roseola efflorescence skin_rash blizzard foolhardy heady reckless', 'spots musca_volitans muscae_volitantes floater topographic_point place spot point smudge blot daub smear smirch slur speckle dapple patch fleck maculation position post berth office billet situation touch bit pip spotlight stain descry espy spy recognize recognise distinguish discern pick_out make_out tell_apart blemish blob', 'blisters blister bulla bleb vesicate scald whip', 'clouding clouding_up overcast cloud obscure befog becloud obnubilate haze_over fog mist defile sully corrupt taint mottle dapple', 'mouth oral_cavity oral_fissure rima_oris mouthpiece sass sas

CLEAN SYNONYMS:  ['tenderswollen', 'breats', 'fatigue weariness tiredness fatigue duty tire pall weary jade wear upon tire out wear wear out outwear wear down fag out fag', 'slight rebuff coldshoulder little flimsy fragile tenuous thin slender slim svelte', 'bleeding hemorrhage haemorrhage shed blood bleed leech phlebotomize phlebotomise run', 'cramping cramp hamper halter strangle', 'missing miss lose neglect pretermit omit drop leave out overlook overleap lack escape lacking absent wanting', 'period time period period of time geological period menstruation menses menstruum catamenia flow point full stop stop full point', 'bloating bloat', 'constipation irregularity stultification impairment deadening', 'crying weeping tears cry outcry call yell shout vociferation war cry rallying cry battle cry watchword shout out scream holler hollo squall weep exclaim cry out call out blazon out clamant exigent insistent instant egregious flagrant glaring gross rank', 'gas gaseous state gasoline ga

WORD_LIST: ['diarrhea', 'fever', 'fatigue', 'abdominal pain', 'cramping', 'blood in your stool', 'mouth sores', 'reduced appetite', 'weight loss', 'anus pain', ' drainage near anus']
WORD_LIST_NEW: ['diarrhea', 'fever', 'fatigue', 'abdominal pain', 'cramping', 'blood your stool', 'mouth sores', 'reduced appetite', 'weight loss', 'anus pain', 'drainage near anus']
joined_symps: diarrhea fever fatigue abdominal pain cramping blood your stool mouth sores reduced appetite weight loss anus pain drainage near anus
SYNONYMS: ['diarrhea diarrhoea looseness_of_the_bowels looseness', 'fever febrility febricity pyrexia feverishness', 'fatigue weariness tiredness fatigue_duty tire pall weary jade wear_upon tire_out wear wear_out outwear wear_down fag_out fag', 'abdominal abdominal_muscle ab', 'pain hurting painfulness pain_sensation painful_sensation pain_in_the_neck nuisance annoyance bother botheration infliction pain_in_the_ass trouble ail anguish hurt', 'cramping cramp hamper halter strangle',

In [6]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    print("TOKENIZED WORDS: ",pattern_words)
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

TOKENIZED WORDS:  ['fluid runny', 'horn in intrud nos nozzl nuzzl olfact org pok pry scent wind', 'airless clos stodgy stuffy unair', 'in nos nuzzl org pry wind', 'afflict huffy mad pain raw sensit sor tend', 'pharynx throat', 'cough', 'congest overcrowd', 'body consist dead eubst person phys soundbox structure torso trunk', 'ach hurt lanct pin smart suff yearn yen', 'sneez sternut']
TOKENIZED WORDS:  ['cent ey eyebal heart middl ocul opt', 'aggrav annoy both discomfort excit innerv irrit piqu provoc temp vex', 'runny', 'clos stuffy', 'in nos nuzzl org pry wind', 'bouff gusty intumesc puffy tumesc tumid turgid', 'reek washy watery weak', 'ey heart ocul', 'sternut', 'ablaz conflagr enkindl fir heat ignit inflam kindl red stir up wak', 'antsy fidg fret itchy', 'in nos nuzzl org pry wind', 'fidg itchy', 'throat']
TOKENIZED WORDS:  ['red rub', 'ey heart ocul', 'bloodr bolshevik bolshy carmin cer cherry cherryr crimson flush ink loss marx redfac riv ruby rubyr ruddy scarlet viol', 'ey heart

In [9]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 100)
net = tflearn.fully_connected(net, 50)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=5000, batch_size=16, show_metric=True)
model.save('model.tflearn')

Training Step: 9999  | total loss: 0.62886 | time: 0.008s
| Adam | epoch: 5000 | loss: 0.62886 - acc: 0.9139 -- iter: 16/28
Training Step: 10000  | total loss: 1.15537 | time: 0.012s
| Adam | epoch: 5000 | loss: 1.15537 - acc: 0.8226 -- iter: 28/28
--
INFO:tensorflow:C:\Users\Steven\Documents\HTN PROJECT 2\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [8]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y, 'classes_details':classes_details}, open( "training_data", "wb" ) )